In [ ]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt

print(f'pd=={pd.__version__}')

In [ ]:
# read in train and test datasets
train_transaction = pd.read_csv('/Users/oskarwallberg/desktop/kaggle-datasets/ieee-fraud-detection/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('/Users/oskarwallberg/desktop/kaggle-datasets/ieee-fraud-detection/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('/Users/oskarwallberg/desktop/kaggle-datasets/ieee-fraud-detection/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('/Users/oskarwallberg/desktop/kaggle-datasets/ieee-fraud-detection/test_identity.csv', index_col='TransactionID')

test_transaction.columns = train_transaction.columns.drop(labels='isFraud')
test_identity.columns = train_identity.columns

train_transaction.shape, test_transaction.shape, train_identity.shape, test_identity.shape

### The point of this notebook
---
=> To group Transactions into client transaction histories. Key features for grouping are card1, addr1 and D1 however these do not partition the dataset perfectly into distinct clients. Thus some further analysis is required, such as comparing aggregated measures of known variables to other measures known to be true for a single client. 

Some examples:
* Measures of clients that stay constant for each transaction of that client will have a standard deviation of 0 when aggregated, if not we know the group consists of two or more clients and must be split further. 
* Aggregated measure of clients that always increment (for example counter of client transactions) should have nunique values equal to the number of transactions in that group. If not we know the group consists of two or more clients.

This is not a perfect method as the dataset contains multiple NaNs which disturb the aggregated measures. This can however be somewhat accounted for by replacing NaNs with large negative values (e.g. -999) and make it clear that a NaN values exists within that group. NaN values can also be left untouched however that can have consequences for the grouping, essentially making some groups dissapear.

In [ ]:
# concatenate train and test datasets to one full dataset

train = train_transaction.merge(right=train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(right=test_identity, how='left', left_index=True, right_index=True)

dataset = pd.concat([train, test])
dataset.shape

In [ ]:
# add labels for datapoints belonging to test dataset
dataset['isTest'] = np.r_[np.zeros(train.shape[0]), np.ones(test.shape[0])]

In [ ]:
dataset['D1'] # D1: days since card was registered and opened for use

In [ ]:
dataset['TransactionDay'] = dataset['TransactionDT'] // (24*60*60) # TransactionDay: Timedelta in days since transactions began being stored for this dataset
dataset['D1n'] = dataset['TransactionDay'] - dataset['D1'] # D1n: The day the card was registered (measured from point when this dataset was starting being collected)
dataset['D2n'] = dataset['TransactionDay'] - dataset['D2']
dataset['D4n'] = dataset['TransactionDay'] - dataset['D4']
dataset['D10n'] = dataset['TransactionDay'] - dataset['D10']
dataset['D15n'] = dataset['TransactionDay'] - dataset['D15']

In [7]:
columns_of_interest = ['D1n', 'D2n', 'D4n', 'D10n', 'D15n', 'C13', 'card1', 'addr1', 'TransactionAmt', 'dist1']
dataset[columns_of_interest].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1097231 entries, 2987000 to 4170239
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   D1n             1089931 non-null  float64
 1   D2n             581665 non-null   float64
 2   D4n             851458 non-null   float64
 3   D10n            1008664 non-null  float64
 4   D15n            996049 non-null   float64
 5   C13             1092483 non-null  float64
 6   card1           1097231 non-null  int64  
 7   addr1           965916 non-null   float64
 8   TransactionAmt  1097231 non-null  float64
 9   dist1           453743 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 92.1 MB


In [8]:
dataset[columns_of_interest] = dataset[columns_of_interest].fillna(value=-999)
dataset[columns_of_interest].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1097231 entries, 2987000 to 4170239
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   D1n             1097231 non-null  float64
 1   D2n             1097231 non-null  float64
 2   D4n             1097231 non-null  float64
 3   D10n            1097231 non-null  float64
 4   D15n            1097231 non-null  float64
 5   C13             1097231 non-null  float64
 6   card1           1097231 non-null  int64  
 7   addr1           1097231 non-null  float64
 8   TransactionAmt  1097231 non-null  float64
 9   dist1           1097231 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 92.1 MB


In [9]:
# group (client) ids
dataset['uid'] = dataset.groupby(['card1', 'addr1', 'D1n']).ngroup()
dataset['uid']

TransactionID
2987000    289126
2987001     38068
2987002     78964
2987003    379675
2987004     76142
            ...  
4170235    287460
4170236     50451
4170237    353330
4170238    348860
4170239     95476
Name: uid, Length: 1097231, dtype: int64

In [10]:
dataset['uid'].nunique()

386568

In [11]:
# add rowCount feature to sum and count the number of rows in each aggregated group
dataset['rowCount'] = np.ones(dataset.shape[0])

In [12]:
# filter away all uid associated with a single transaction, uids with only one transaction must be a single client
uid_counts = dataset['uid'].value_counts()
uid_2p = uid_counts.loc[uid_counts >= 2].index # 2p meaning "2 plus"
dataset_2p = dataset.loc[dataset['uid'].isin(uid_2p)].copy()
dataset_2p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 874654 entries, 2987002 to 4170237
Columns: 442 entries, isFraud to rowCount
dtypes: float64(407), int64(4), object(31)
memory usage: 2.9+ GB


In [20]:
dataset_clients: pd.DataFrame = dataset_2p.groupby(['uid']).aggregate({
    'D4n': [('D4n_mean', 'mean'),
            ('D4n_std', 'std')],
    'D10n': [('D10n_mean', 'mean'),
             ('D10n_std', 'std')],
    'D15n': [('D15n_mean', 'mean'),
             ('D15n_std', 'std')],
    'TransactionAmt': [('TrAmt_mean', 'mean'),
                       ('TrAmt_std', 'std')],
     'dist1': [('dist1_mean', 'mean'),
               ('dist1_std', 'std')],
     'C13': [('C13_nunq', 'nunique')],
     'rowCount': [('rowCount_sum', 'sum')]
})
dataset_clients.columns = dataset_clients.columns.droplevel(level=0)
dataset_clients

,D4n_mean,D4n_std,D10n_mean,D10n_std,D15n_mean,D15n_std,TrAmt_mean,TrAmt_std,dist1_mean,dist1_std,C13_nunq,rowCount_sum
uid,,,,,,,,,,,,
1,-299.666667,0.577350,-299.666667,0.577350,-299.666667,0.577350,79.666667,89.494879,-662.333333,583.123772,3,3.0
9,156.000000,0.000000,156.000000,0.000000,156.000000,0.000000,226.000000,0.000000,-999.000000,0.000000,1,2.0
25,339.000000,0.000000,339.000000,0.000000,339.000000,0.000000,83.613000,0.000000,-999.000000,0.000000,1,2.0
28,-356.333333,556.565659,105.666667,243.641814,-35.000000,0.000000,62.966667,34.797282,-999.000000,0.000000,3,3.0
32,-999.000000,0.000000,-999.000000,0.000000,-999.000000,0.000000,75.000000,0.000000,-999.000000,0.000000,2,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
386559,97.000000,0.000000,111.500000,20.506097,97.000000,0.000000,232.470000,0.000000,-999.000000,0.000000,2,2.0
386561,106.642857,77.846602,106.642857,77.846602,59.214286,0.425815,33.092857,13.877803,433.285714,606.801124,12,14.0
386562,357.250000,0.500000,22.250000,0.500000,22.250000,0.500000,124.750000,71.658333,-240.750000,505.500000,4,4.0
